<a href="https://colab.research.google.com/github/tractorjuice/Building_BoK/blob/main/Building_Wardley_Mapping_Body_of_Knowledge_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Body of Knowledge using Pinecone, Langchain and OpenAI
## Part 2, transcribe the audio files

This example shows how to create and query an internal knowledge base using ChatGPT.

This requires a GPU/TPU runtime.

## Runtime Checks

### Check we are running on a GPU/TPU and check the available memory

In [ ]:
try:
  gpu_info = !nvidia-smi
except:
  print('No GPU')
else:
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## Set Up


###Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Part 2 - Transcribe the audio files
*(Skip to next section to load data store from files if it has been saved locally to save cost of embeddings)*

## Check audio files are stored on Google Drive

In [ ]:
import os

DOCS_FOLDER = "/content/gdrive/MyDrive/WardleyKB"  # Google drive folder to save the audio clips from YouTube videos
AUDIO_FOLDER = os.path.join(DOCS_FOLDER, "audio")  # Sub-directory for audio files

# Check if directory exists and if not, create it
if not os.path.exists(DOCS_FOLDER):
    os.makedirs(DOCS_FOLDER)

# Check if sub-directory for audio exists and if not, create it
if not os.path.exists(AUDIO_FOLDER):
    os.makedirs(AUDIO_FOLDER)


## Use OpenAI Whisper to convert to text

In [ ]:
import jax
jax.devices()

In [ ]:
!pip install jax[tpu] -f https://storage.googleapis.com/jax-releases/libtpu_releases.html!pip install --quiet git+https://github.com/sanchit-gandhi/whisper-jax.git datasets soundfile librosa!pip install --quiet cached_property

In [ ]:
from whisper_jax import FlaxWhisperPipline
import jax.numpy as jnp

#For most GPUs, the dtype should be set to jnp.float16. For A100 GPUs or TPUs, the dtype should be set to jnp.bfloat16:
pipeline = FlaxWhisperPipline("openai/whisper-large-v2", dtype=jnp.bfloat16, batch_size=16)

In [ ]:
from jax.experimental.compilation_cache import compilation_cache as cc

cc.initialize_cache("./jax_cache")

In [ ]:
unique_video_ids = []

with open(f'{AUDIO_FOLDER}/videos.txt', 'r') as file:
    for line in file:
        # Remove linebreak which is the last character of the string
        curr_place = line[:-1]
        # Add item to the list
        unique_video_ids.append(curr_place)

### Transcribe the audio files and save the text to the Google drive

In [ ]:
import re, json, os

def replace_wordly_with_wardley(transcription):
    # Case-insensitive replacement of "worldy" and "worldly" with "Wardley"
    for word in ["wordly", "worldly"]:
        pattern = re.compile(word, re.IGNORECASE)
        text = transcription['text']
        text = pattern.sub("Wardley", text)
        transcription['text'] = text
        for chunk in transcription['chunks']:
            chunk_text = chunk['text']
            chunk_text = pattern.sub("Wardley", chunk_text)
            chunk['text'] = chunk_text
    return transcription

def transcribe_file(filename):
    print (f"Transcribing New file: {filename}")
    transcription = pipeline(filename, return_timestamps=True)
    transcription = replace_wordly_with_wardley(transcription)
    print(transcription)
    return transcription

# make sure the transcripts directory exists
os.makedirs(os.path.join(AUDIO_FOLDER, 'transcripts'), exist_ok=True)

transcriptions = []
total_videos = len(unique_video_ids)

for counter, video in enumerate(unique_video_ids, start=1):
    transcript_filename = f'{AUDIO_FOLDER}/transcripts/{video}_large.txt'
    audio_filename = f'{AUDIO_FOLDER}/clips/{video}.webm'
    print(f"{counter} of {total_videos}")

    if not os.path.isfile(transcript_filename):
        if os.path.isfile(audio_filename):
            transcription = transcribe_file(audio_filename)
            transcriptions.append(transcription)
            with open(transcript_filename, 'w') as f:
                f.write(json.dumps(transcription))
        else:
            print (f"File does not exist: {audio_filename}")
    else:
        print(f"Existing File: {transcript_filename}")
        with open(transcript_filename, 'r') as f:
            transcription = json.load(f)
        transcription = replace_wordly_with_wardley(transcription)
        print(transcription)
        with open(transcript_filename, 'w') as f:
            f.write(json.dumps(transcription))
